- idiom 관용구
    - 특정 작업을 수행하기 위해 코드를 작성하는 특별한 방법
    - 매번 동일한 구조를 반복하고 따르는 것이 일반적
    - 디자인 패턴과는 다름
    - 언어에 의존적: 파이썬의 관용구를 따른 코드를 pythonic 파이썬 스럽다고 표현
- 파이썬스러운 코드를 작성하는 이유
    - 일반적으로 더 나은 성능을 낸다
    - 코드가 더 작고 이해하기 쉽다
    - 동일한 패턴과 구조를 가지면 전체 개발팀이 실수를 줄이고 문제의 본질에 보다 집중할 수 있게 된다
- 이 장의 목표
    - 인덱스와 슬라이스를 이해하고 인덱싱 가능한 객체를 올바른 방식으로 구현하기
    - 시퀀스와 이터러블 구현하기
    - 컨텍스트 관리자를 만드는 모범 사례 연구 그리고 어떻게 효율적으로 작성할 수 있는지
    - 매직 메서드를 사용해 보다 관용적인 코드 구현
    - 파이썬에서 부작용을 유발하는 흔한 실수 피하기

# 인덱스와 슬라이스
- 파이썬의 인덱스
    - 0부터 시작하는 건 일반적
    - 음수 인덱스 사용 가능
        ```python
        >>> my_numbers = (4, 5, 3, 9)
        >>> my_numbers[-1]
        9
        >>> my_numbers[-3]
        5
        ```
    - slice를 사용하여 특정 구간의 요소를 구할 수 있음
        ```python
        >>> my_numbers = (1, 1, 2, 3, 5, 8, 13, 21)
        >>> my_numbers[2:5]
        (2, 3, 5)
        ```
        - 주의: 시작 인덱스는 포함, 끝 인덱스는 제외
    - 시작, 끝, 또는 간격 파라미터 중 하나를 제외할 수 있음
        ```python
        >>> my_numbers[:3]
        (1, 1, 2)
        >>> my_numbers[3:]
        (3, 5, 8, 13, 21)
        >>> my_numbers[::] # my_numbers[:] 도 마찬가지로 복사본을 만든다
        (1, 1, 2, 3, 5, 8, 13, 21)
        >>> my_numbers[1:7:2]
        (1, 3, 8)
        ```
    - slice 는 내장 객체이므로 직접 호출할 수 있다.
        ```python
        >>> interval = slice(1, 7, 2)
        >>> my_numbers[interval]
        (1, 3, 8)
        >>> interval = slice(None, 3)
        >>> my_numbers[interval] == my_numbers[:3]
        True
        ```
> 💡튜플, 문자열, 리스트의 특정 요소를 가져오려고 한다면 for 루프를 도는 것보다 slice를 사용하자

## 자체 시퀀스 생성
- 매직 메서드
    - 파이썬에서 특수한 동작을 수행하기 위해 예약한 메서드, 이중 밑줄로 둘러싸여 있다
    - 인덱싱 기능은 `__getitem__` 매직 메서드 덕분에 동작
        - `myobject[key]` 같은 형태로 사용할때 호출되는 메서드
        - key를 `__getitem__(key)` 형태로 파라미터로 전달
- 시퀀스
    - `__getitem__` 과 `__len__` 을 모두 구현한 객체
    - 매직 메서드 덕분에 반복이 가능함
    - 리스트, 튜플., 문자열이 시퀀스 객체의 예
- 자체 시퀀스를 파이썬 스럽게 구현하려면 어떻게 해야할까?

In [1]:
from collections.abc import Sequence

class Items(Sequence):
    def __init__(self, *values):
        self._values = list(values)

    def __len__(self):
        return len(self._values)
    
    def __getitem__(self, item):
        return self._values.__getitem__(item)

In [4]:
items = Items([1, 2, 3, 4, 5])
print(len(items))
print(items[0], items[-1])
print(items[2:4])

5
1 5
[3, 4]


- collections.abc의 Sequence 인터페이스를 구현
    - 이 인터페이스를 상속받으면 해당 클래스가 어떤 클래스인지 바로 알 수 있음
    - 필요한 요건들을 꼭 구현해야 하는 강제성을 띄게 됨
- 세 가지 방법
    - 컴포지션: list 클래스를 상속받은게 아니라 구현체인 values를 가지고 있음 (위의 방법)
    - 상속: collections.UserList 부모 클래스를 상속받을 수 있음
        - 나중에 주의사항 이야기할 것임!
    - 나만의 시퀀스를 구현할 수도 있음
        - 범위로 인덱싱하는 결과는 해당 클래스와 같은 타입의 인스턴스여야 함
            - 튜플을 인덱싱하면 튜플, 문자열을 인덱싱하면 문자열! (근데 위에서는 왜? 🤔)
        - slice에 의해 제공된 범위는 마지막 요소는 제외!

# 컨텍스트 관리자 context manager
- 패턴을 구현할 때 유용하다
    - 사전 조건과 사후 조건이 있는 코드를 실행해야 할 때 사용할 수 있는 훌륭한 도구!
    - 리소스 관리와 관련하여 자주 볼 수 있다
        - 파일을 열면 파일 디스크립터 누수를 막기 위해 파일이 끝나고 적절히 닫히도록 구현
        - 서비스나 소켓 연결을 나중에 적절히 닫거나 임시 파일을 제거하도록 구현

In [12]:
!echo 'Spam Spam Spam Egg Spam' > menu.txt

In [13]:
# some way
fd = open('menu.txt')
try:
    data = fd.read()
finally:
    fd.close()

print(data)

Spam Spam Spam Egg Spam



In [14]:
# pythonic way
with open('menu.txt') as fd:
    data = fd.read()
    print(data)

Spam Spam Spam Egg Spam



- PEP-343
    - with 문은 컨텍스트 관리자로 진입하게 한다
    - open 함수가 컨텍스트 관리자 프로토콜을 구현한다.
    - 위에서는 예외가 발생한 경우에도 블록이 완료되면 파일이 자동으로 닫히게끔 구현되어 있다.
- 컨텍스트 관리자의 구성
    - `__enter__` 과 `__exit__`
    - with 문에서 `__enter__` 를 호출하고 리턴 값을 `as`에 할당
        - 근데 as 가 꼭 필요한 것은 아님
    - with 문이 끝나면 `__exit__` 을 호출함
        - 예외나 오류가 있어도 여전히 호출됨
        - 예외가 발생한 경우 예외를 파라미터로 받기 때문에 임의의 방법으로 내부에서 처리할 수도 있음
- 독립적으로 유지되어야 하는 코드를 분리할 때 좋은 방법
- 예시: 데이터베이스 백업
    - 백업은 오프라인 상태에서만 해야 함
    - 백업 중 서비스 중지, 백업이 끝나면 성공했든 실패했든 프로세스 다시 시작해야 함

In [4]:
# some way

def stop_database():
    print('systemctl stop postgresql')

def start_database():
    print('systemctl start postgresql')

class DBHandler:
    def __enter__(self):
        stop_database()
        return self

    def __exit__(self, exc_type, ex_value, ex_traceback):
        start_database()

    def db_backup(self):
        print('pg_dump database')

def main():
    with DBHandler() as db:
        db.db_backup()

main()

systemctl stop postgresql
pg_dump database
systemctl start postgresql


- DBHandler
    - `__enter__` 의 반환 값은 쓸모 없지만 일반적으로 무언가를 반환하는게 좋은 습관
    - `main()` 에서 유지보수 작업과 상관 없이 백업을 실행, 백업에 오류가 있어도 여전히 `__exit__` 을 호출
    - `__exit__` 메서드는 일반적으로 아무것도 반환하지 않지만, True 를 반환하면 잠재적으로 발생한 예외를 호출자에게 전파지 않고 멈춘다는 것을 뜻함
        - 좋지 않은 습관!
        - 실수로 True 를 리턴하지 않도록 하자

## 컨텍스트 관리자 구현
- 컨텍스트 관리자를 좀 더 간결하게, 보다 쉽게 구현하는 방법이 있다
- `contextlib` 모듈을 사용하자!

In [5]:
# more pythonic way?

import contextlib

@contextlib.contextmanager
def db_handler():
    try:
        stop_database()
        yield
    finally:
        start_database()


def db_backup():
    print("pg_dump database")


with db_handler() as db:
    db_backup()

systemctl stop postgresql
pg_dump database
systemctl start postgresql


- generator 함수 (yield 참조!) 를 정의하고 @contextlib.contextmanager 데코레이터를 적용하였다
    - 데코레이터를 적용하면 yield 문 앞의 모든 것이 `__enter__` 메서드의 일부처럼 취급됨
    - yield 문의 반환값 (여기서는 아무것도 반환하지 않지만! = None) 은 as x: <- 에 들어감
    - yield 문 다음의 모든 것은 `__exit__` 로직으로 취급됨
- 장점
    - 기존 함수를 리팩토링하기 쉬움
        - 객체지향적으로 의미를 가지지 않는 가짜 부모 클래스를 만들지 않아도 됨!
        - 매직 메서드를 추가하면 업무 도메인에 더 얽히게 되고, 책임이 커지고, 더 많은 걸 지원해야 될지도...

In [6]:
# another pythonic way
import contextlib


class dbhandler_decorator(contextlib.ContextDecorator):
    def __enter__(self):
        stop_database()
        return self

    def __exit__(self, exc_type, ex_value, ex_traceback):
        start_database() 


@dbhandler_decorator()
def offline_backup():
    print('pg_dump database')


offline_backup()

systemctl stop postgresql
pg_dump database
systemctl start postgresql
